# Distribución Metros Cuadrados Construidos


In [3]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    if nombre.startswith("df_") and "alfa" in nombre and "puebla" in nombre and 'q' not in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_agosto_2024_puebla',
 'df_alfa_julio_2024_puebla',
 'df_alfa_junio_2024_puebla',
 'df_alfa_septiembre_2024_puebla']

In [4]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    # Verificar si la columna 'm2_contruido' existe en el DataFrame
    
        # Renombrar la columna
    df.rename(columns={'m2_contruido': 'm2_construido','segmento':'categoria','Categoria':'categoria','metros_construido':'m2_construido','Precio':'precio'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [5]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
dataframes_name = []

for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id', 'categoria','m2_construido','precio']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)
     # Componer el nombre del DataFrame en la lista dataframes_name
    dataframes_name.append(nombre_df)


In [6]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return None  # En caso de que el precio no caiga en ningún rango (caso raro)

In [7]:
for df in dataframes_list:    
    df = df.drop(columns=['categoria'])
    # Asignar la categoría real a cada registro
    df['categoria'] = df['precio'].apply(asignar_categoria)

# Cantidad de registros por metros cuadrados construidos, por segmento

In [8]:
def procesar_dataframes(dataframes_list, dataframes_name):
    for i, df in enumerate(dataframes_list):
        # Crear una nueva columna con la primera letra de la categoría
        df['segmento'] = df['categoria'].str[0]
        df.drop(columns=['categoria'], inplace=True)

        # Clasificar en rangos específicos
        bins = [0, 100, 150, 200, 250, 300, float('inf')]
        labels = ['0-100', '100-150', '150-200', '200-250', '250-300', '300+']
        df['m2_construido'] = pd.cut(df['m2_construido'], bins=bins, labels=labels, right=False)

        # Agrupar y contar los datos
        tabla = df.groupby(['segmento', 'm2_construido']).size().unstack(fill_value=0)

        # Imprimir el nombre del DataFrame y la tabla resultante
        print(f"Nombre del DataFrame: {dataframes_name[i]}")
        print(tabla)

# Procesar los DataFrames de la lista
procesar_dataframes(dataframes_list, dataframes_name)

Nombre del DataFrame: df_alfa_agosto_2024_puebla
m2_construido  0-100  100-150  150-200  200-250  250-300  300+
segmento                                                      
A                  2       37       44      129       27    31
B                  4       83       83       61        9    19
C                 43      179       68       28       19    23
D                 97      124       65       36       39    51
E                161      243      158      191       92   233
L                  0        2        0        1        2   151
S                  1       27       29       89      115   439
Nombre del DataFrame: df_alfa_julio_2024_puebla
m2_construido  0-100  100-150  150-200  200-250  250-300  300+
segmento                                                      
A                 26       97       68      131       27    42
B                 35      205       91       64       10    30
C                188      250       73       29       21    40
D                256 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_9592\2613299638.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['segmento'] = df['categoria'].str[0]
C:\Users\yoe11\AppData\Local\Temp\ipykernel_9592\2613299638.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['categoria'], inplace=True)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_9592\2613299638.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

## Tabla general de frecuencia
 Recuento total de filas para cada combinación de valores únicos de 'segmento' y 'm2_contruido' con todos los DataFrame

In [9]:
# Crear una nueva función para sumar las tablas de todos los DataFrames
def sumar_tablas(dataframes_list):
    # Inicializar una tabla vacía para almacenar la suma
    tabla_suma = None
    
    # Iterar sobre cada DataFrame en la lista
    for df in dataframes_list:
        # Agrupar y contar los datos en el DataFrame actual
        tabla_actual = df.groupby(['segmento', 'm2_construido']).size().unstack(fill_value=0)
        
        # Sumar la tabla actual al acumulador (tabla_suma)
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    # Imprimir la tabla resultante de la suma
    print("Tabla general de frecuencia:")
    print(tabla_suma)

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
sumar_tablas(dataframes_list)

Tabla general de frecuencia:
m2_construido  0-100  100-150  150-200  200-250  250-300  300+
segmento                                                      
A                 83      325      258      520      109   127
B                110      663      367      222       34    96
C                562      883      279      105       70   115
D                800      556      211      117      110   174
E                853      775      512      650      277   767
L                  0        6        0        5       10   604
S                 55      264      322      442      472  1707


In [10]:
orden_segmentos=["E", "D", "C", "B","A", "S", "L",'ELITE' ]

def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        tabla_actual = df.groupby(['m2_construido', 'segmento']).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    fig = graficar_tabla(tabla_suma)
    
    # Guardar la gráfica como archivo HTML
    guardar_grafico_como_html(fig, 'g_bar_dist_m2_construidos', carpeta='assets/graficas')
    
    return fig

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

def graficar_tabla(tabla):  
    segmentos = tabla.index
     # Reordenar segmentos según el orden definido
    segmentos_ordenados = [seg for seg in orden_segmentos if seg in segmentos]
    
    banos = tabla.columns
    
    fig = go.Figure()
    
    for i, segmento in enumerate(segmentos):
        color = colores[i % len(colores)]
        fig.add_trace(go.Bar(
            x=banos,
            y=tabla.loc[segmento],
            name=segmento,
            marker_color=color
        ))
    
    fig.update_layout(
        xaxis_title="Segmento",        
        yaxis=dict(
            title='Número de casas',
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
            gridwidth=1,
        ),
        legend_title="Rango de M2",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10) # Ajusta los márgenes (left, right, top, bottom)
    )    
    
    fig.show()    
    return fig

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    ruta_archivo = os.path.join(carpeta, f'{nombre_archivo}.html')
    pio.write_html(fig, ruta_archivo)
# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
fig = sumar_tablas(dataframes_list)

Tabla general de frecuencia:
segmento         A    B    C    D    E    L     S
m2_construido                                    
0-100           83  110  562  800  853    0    55
100-150        325  663  883  556  775    6   264
150-200        258  367  279  211  512    0   322
200-250        520  222  105  117  650    5   442
250-300        109   34   70  110  277   10   472
300+           127   96  115  174  767  604  1707


In [14]:
# Definir el orden deseado de los segmentos
orden_segmentos = ["E", "D", "C", "B", "A", "S", "L", ]#'ELITE']

def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        # Agrupar y contar casas según 'm2_construido' y 'segmento'
        tabla_actual = df.groupby(['segmento', 'm2_construido']).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    
    # Reordenar los índices (segmentos) de acuerdo a 'orden_segmentos'
    tabla_suma = tabla_suma.reindex(orden_segmentos).fillna(0)
    
    # Graficar la tabla reordenada
    fig = graficar_tabla(tabla_suma)
    
    # Guardar la gráfica como archivo HTML
    guardar_grafico_como_html(fig, 'g_bar_dist_m2_construidos', carpeta='assets/graficas')
    
    return fig

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

def graficar_tabla(tabla):  
    # Obtenemos las columnas (m2_construido) y los índices (segmentos) de la tabla
    segmentos = tabla.index
    m2_construido = tabla.columns
    
    fig = go.Figure()
    
    # Trazar las barras para cada rango de m2_construido dentro de cada segmento
    for i, m2 in enumerate(m2_construido):
        color = colores[i % len(colores)]
        fig.add_trace(go.Bar(
            x=segmentos,            # Segmentos en el eje X
            y=tabla[m2],            # Cantidad de casas para ese rango de m2_construido
            name=f"{m2}",        # Nombre del rango de m2_construido
            marker_color=color
        ))
    
    fig.update_layout(
        xaxis_title="Segmento",        
        yaxis=dict(
            title='Número de casas',
            gridcolor='#dddcda',  # Color de las líneas que dividen los rangos del eje Y
            gridwidth=1,
        ),
        legend_title="Rango M2 construido",
        barmode='group',  # Las barras se agrupan por rango de m2_construido
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10)  # Ajusta los márgenes (left, right, top, bottom)
    )    
    
    fig.show()    
    return fig

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    ruta_archivo = os.path.join(carpeta, f'{nombre_archivo}.html')
    pio.write_html(fig, ruta_archivo)

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
fig = sumar_tablas(dataframes_list)


Tabla general de frecuencia:
m2_construido  0-100  100-150  150-200  200-250  250-300  300+
segmento                                                      
A                 83      325      258      520      109   127
B                110      663      367      222       34    96
C                562      883      279      105       70   115
D                800      556      211      117      110   174
E                853      775      512      650      277   767
L                  0        6        0        5       10   604
S                 55      264      322      442      472  1707


## GRAL

In [13]:
def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        tabla_actual = df.groupby(['m2_construido', 'segmento']).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    
    fig = graficar_tabla(tabla_suma)
    
    # Guardar la gráfica como archivo HTML
    guardar_grafico_como_html(fig, 'g_pie_dist_m2_construidos', carpeta='assets/graficas')
    
    return fig

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

def graficar_tabla(tabla):
    # Sumar los valores por 'm2_contruido' para obtener el total por rango de M2
    suma_por_m2 = tabla.sum(axis=1)
    
    fig = go.Figure(data=[go.Pie(
        labels=suma_por_m2.index.astype(str),
        values=suma_por_m2.values,
        marker=dict(colors=colores),
        textinfo='percent',
        insidetextorientation='radial'
    )])
    
    fig.update_layout(
        # title="Distribución de M2 construidos",
        legend_title="Rango M2 Construido",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02, 
            xanchor='right',
            x=0.72
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10) # Ajusta los márgenes (left, right, top, bottom)
    )    
    
    fig.show()
    return fig

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    ruta_archivo = os.path.join(carpeta, f'{nombre_archivo}.html')
    pio.write_html(fig, ruta_archivo)

fig1 = sumar_tablas(dataframes_list)

Tabla general de frecuencia:
segmento         A    B    C    D    E    L     S
m2_construido                                    
0-100           83  110  562  800  853    0    55
100-150        325  663  883  556  775    6   264
150-200        258  367  279  211  512    0   322
200-250        520  222  105  117  650    5   442
250-300        109   34   70  110  277   10   472
300+           127   96  115  174  767  604  1707
